### Exogenous Example
## Authored by BlockScience, MV Barlin
### Updated July-10-2019 


Key assumptions and space:
1. Implementation of System Model in cell 2
2. Timestep = day
3. Launch simulation, without intervention from changing governance policies

#### Library Imports

In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
#from tabulate import tabulate
from scipy import stats
sns.set_style('whitegrid')
from decimal import Decimal
from datetime import timedelta

%matplotlib inline

## cadCAD Setup
#### ----------------cadCAD LIBRARY IMPORTS------------------------

In [2]:
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
#from simulations.validation import sweep_config
from cadCAD import configs
from cadCAD.configuration import append_configs
from cadCAD.configuration.utils import proc_trigger, ep_time_step, config_sim

In [3]:
#from cadCAD.configuration.utils.parameterSweep import config_sim

In [4]:
from typing import Dict, List

#### ----------------Random State Seed-----------------------------

In [5]:
seed = {
#    'z': np.random.RandomState(1)
}

#### Timestamp

In [6]:
ts_format = '%Y-%m-%d %H:%M:%S'
t_delta = timedelta(days=0, minutes=0, seconds=1)
def set_time(_g, step, sL, s, _input):
    y = 'timestamp'
    x = ep_time_step(s, dt_str=s['timestamp'], fromat_str=ts_format, _timedelta=t_delta)
    return (y, x)

# ASSUMED PARAMETERS

### PRICE LIST

In [7]:
# dai_xns_conversion = 1.0 # Assumed for static conversion 'PUBLISHED PRICE LIST' DEPRECATED

#### Initial Condition State Variables

In [8]:
del_stake_pct = 2

starting_xns = float(10**10)   # initial supply of xns tokens
starting_broker_xns = float(1 * 10**8) # inital holding of xns token by broker app
starting_broker_fiat = float(1 * 10**5) # inital holding of xns token by broker app
starting_broker_stable = float(1 * 10**6) # inital holding of stable token by broker app
starting_deposit_acct = float(100) # inital deposit locked for first month of resources TBD: make function of resource*price
starting_entrance = float(1 * 10**4) # TBD: make function of entrance fee % * cost * # of initial apps
starting_app_usage = float(10) # initial fees from app usage 
starting_platform = float(100) # initial platform fees  
starting_resource_fees = float(10) # initial resource fees usage paid by apps  
starting_app_subsidy = float(0.25* 10**9) # initial application subsidy pool
starting_stake = float(4 * 10**7)
starting_stake_pool = starting_stake + ((3*10**7)*(del_stake_pct))  # initial staked pool + ((3*10**7)*(del_stake_pct))

#starting_block_reward = float(0) # initial block reward   MOVED ABOVE TO POLICY
starting_capacity_subsidy = float(7.5 * 10**7)  # initial capacity subsidy pool
starting_delegate_holdings = 0.15 * starting_xns


#### Initial Condition Composite State Variables

In [9]:
# subsidy limit is 30% of the 10B supply
starting_treasury = float(5.5 * 10**9) 
starting_app_income = float(0) # initial income to application
starting_resource_income = float(0) # initial income to application
starting_delegate_income = float(0) # initial income to delegate

#### Initial Condition Exogoneous State Variables 

In [10]:
starting_xns_fiat = float(0.01) # initial xns per fiat signal
starting_fiat_ext = float(1) # initial xns per fiat signal
starting_stable_ext = float(1) # initial stable signal

### Exogenous Price Updates

In [11]:
def delta_price(mean,sd):
    '''Returns normal random variable generated by first two central moments of price change of input ticker'''
    rv = np.random.normal(mean, sd)
    return rv

In [12]:

def xns_ext_update(_g, step, sL, s, _input):
    key = 'XNS_fiat_external'
    
    value = s['XNS_fiat_external'] * (1 + delta_price(0.000000, 0.005))
    
    return key, value

From Currency Analysis of DAI-USD pair  
May-09-2018 through June-10-2019  
Datasource: BitFinex  
Analysis of daily return percentage performed by BlockScience

In [13]:
DAI_mean = 0.0000719
DAI_sd = 0.006716

The daily return is computed as:  
$$ r = \frac{Price_n - Price_{n-1}}{Price_{n-1}} $$  
Thus, the modelled current price can be as:  
$$ Price_n = Price_{n-1} * r + Price_{n-1} $$

In [14]:

def stable_update(_g, step, sL, s, _input):
    key = 'stable_external'
    
    value = s['stable_external'] * (1 + delta_price(DAI_mean, DAI_sd))
    return key, value


#### Assumed Parameters

In [15]:
apps_deployed = 1 # Make part of test- application deployment model

starting_deposit_acct = float(100) # inital deposit locked for first month of resources TBD: make function of resource*price

app_resource_fee_constant  =  10**1 # in STABLE, assumed per day per total nodes 
platform_fee_constant = 10 # in XNS
# ^^^^^^^^^^^^ MAKE A PERCENTAGE OR FLAT FEE as PART of TESTING

In [16]:

alpha = 100  # Fee Rate
beta = 0.10  # FIXED Too high because multiplied by constant and resource fees
app_platform = alpha * platform_fee_constant
app_platform

1000

In [17]:
beta_out =beta*100
beta_out

10.0

In [18]:
starting_capacity_subsidy / (5 * 10**7) / 10

0.15

In [19]:

weight = 0.95 # 0.95 internal weight 5% friction from external markets

def xns_int_update(_g, step, sL, s, _input):
    key = 'XNS_fiat_internal'

    internal = s['XNS_fiat_internal']  * weight
    external = s['XNS_fiat_external'] * (1 - weight)
    value = internal + external
    
    return key, value

#### CONFIGURATION DICTIONARY

In [20]:
time_step_count = 3652 # days = 10 years
run_count = 1

#### Genesis States

In [21]:
#----------STATE VARIABLE Genesis DICTIONARY---------------------------
genesis_states = {
    'XNS_fiat_external' : starting_xns_fiat,
    'XNS_fiat_internal' : starting_xns_fiat,
 #   'fiat_external' : starting_fiat_ext,
    'stable_external' : starting_stable_ext,
    'timestamp': '2018-10-01 15:16:24',     #es5
}

In [22]:
#--------------EXOGENOUS STATE MECHANISM DICTIONARY--------------------
exogenous_states =     {
    'XNS_fiat_external' : xns_ext_update,
#     'fiat_external' : starting_fiat_ext,
    'stable_external' : stable_update,
    "timestamp": set_time,
    }

#--------------ENVIRONMENTAL PROCESS DICTIONARY------------------------
env_processes = {
#     "Poisson": env_proc_id
}
#----------------------SIMULATION RUN SETUP----------------------------
sim_config = config_sim(
    {
    "N": run_count,
    "T": range(time_step_count)
#     "M": g  # for parameter sweep
}
)
#----------------------MECHANISM AND BEHAVIOR DICTIONARY---------------
partial_state_update_block = {
    "price": {                 
        "policies": {  
        },
        "variables": {
            'XNS_fiat_internal'     : xns_int_update
#            'app_income'            : app_earn,
        }
    },
}

append_configs(
    sim_configs=sim_config,
    initial_state=genesis_states,
    seeds=seed,
    raw_exogenous_states= exogenous_states,
    env_processes=env_processes,
    partial_state_update_blocks=partial_state_update_block
)

#### Running cadCAD

In [23]:
exec_mode = ExecutionMode()

print("Simulation Execution: Single Configuration")
print()
first_config = configs # only contains config1
single_proc_ctx = ExecutionContext(context=exec_mode.single_proc)
run1 = Executor(exec_context=single_proc_ctx, configs=first_config)
run1_raw_result, tensor_field = run1.main()
result = pd.DataFrame(run1_raw_result)
# print()
# print("Tensor Field: config1")
# print(tabulate(tensor_field, headers='keys', tablefmt='psql'))
# print("Output:")
# print(tabulate(result, headers='keys', tablefmt='psql'))
# print()

Simulation Execution: Single Configuration

single_proc: [<cadCAD.configuration.Configuration object at 0x0000024B3B37AF60>]


C:\Users\mbarl\AppData\Local\Continuum\anaconda3\lib\site-packages\cadCAD\utils\__init__.py:89: FutureWarning: The use of a dictionary to describe Partial State Update Blocks will be deprecated. Use a list instead.
  FutureWarning)


In [24]:
df = result

In [25]:
df.head(10)

,XNS_fiat_external,XNS_fiat_internal,run,stable_external,substep,timestamp,timestep
0,0.010000,0.010000,1,1.000000,0,2018-10-01 15:16:24,0
1,0.009944,0.010000,1,1.000172,1,2018-10-01 15:16:25,1
2,0.009889,0.009997,1,1.003516,1,2018-10-01 15:16:26,2
3,0.009848,0.009992,1,0.990655,1,2018-10-01 15:16:27,3
4,0.009814,0.009985,1,1.001346,1,2018-10-01 15:16:28,4
5,0.009798,0.009976,1,1.002495,1,2018-10-01 15:16:29,5
6,0.009706,0.009967,1,0.994911,1,2018-10-01 15:16:30,6
7,0.009625,0.009954,1,0.998919,1,2018-10-01 15:16:31,7
8,0.009632,0.009938,1,0.995047,1,2018-10-01 15:16:32,8
9,0.009648,0.009922,1,0.980786,1,2018-10-01 15:16:33,9
